## Step 1: Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/MyDrive/ComparisonDetector')
print('Current dir:', os.getcwd())
print('Files:', os.listdir('.')[:10])

## Step 2: Install Dependencies

In [ ]:
!pip install -q tensorflow==2.19.0 numpy opencv-python pillow

## Step 3: Run Training Script

In [ ]:
# Run the multi-cell training script
exec(open('train_keras_multicell.py').read())

## Step 4: Predict on Test Image with Multiple Cells

In [ ]:
import tensorflow as tf
import numpy as np
from PIL import Image, ImageDraw
import os
from configs.config_v2 import ConfigV2
from data.loader_tf2 import build_dataset
from libs.label_dict import get_label_name_map

cfg = ConfigV2()
label_map = get_label_name_map()

# Load best model
best_model_path = os.path.join(cfg.MODEL_DIR, 'best_model_multicell.h5')
if os.path.exists(best_model_path):
    model = tf.keras.models.load_model(best_model_path)
    print(f'✅ Loaded model: {best_model_path}')
else:
    print(f'❌ Model not found: {best_model_path}')
    model = None

if model:
    # Parse one sample from TFRecord to demonstrate
    tfrecord_path = os.path.join(cfg.DATA_DIR, 'train.tfrecord')
    
    def parse_example(example_proto):
        features = {
            'img': tf.io.FixedLenFeature([], tf.string),
            'img_height': tf.io.FixedLenFeature([], tf.int64),
            'img_width': tf.io.FixedLenFeature([], tf.int64),
            'gtboxes_and_label': tf.io.FixedLenFeature([], tf.string),
            'img_name': tf.io.FixedLenFeature([], tf.string),
        }
        parsed = tf.io.parse_single_example(example_proto, features)
        return parsed
    
    dataset = tf.data.TFRecordDataset(tfrecord_path)
    dataset = dataset.map(parse_example)
    
    # Get first sample
    for parsed in dataset.take(1):
        img_name = parsed['img_name'].numpy().decode('utf-8')
        height = int(parsed['img_height'].numpy())
        width = int(parsed['img_width'].numpy())
        img_data = parsed['img'].numpy()
        
        # Decode image
        image = tf.io.decode_raw(img_data, tf.uint8)
        image = tf.reshape(image, [height, width, 3])
        image_np = image.numpy()
        
        # Parse boxes and labels
        gtboxes_and_label = tf.io.decode_raw(parsed['gtboxes_and_label'], tf.int32)
        gtboxes_and_label = tf.reshape(gtboxes_and_label, [-1, 5])
        
        boxes = gtboxes_and_label[:, :4].numpy()
        true_labels = gtboxes_and_label[:, 4].numpy()
        
        print(f"\n🖼️  Image: {img_name} ({width}x{height})")
        print(f"   Contains {len(boxes)} cells")
        print()
        
        # Create output image with predictions
        output_img = Image.fromarray(image_np).convert('RGB')
        draw = ImageDraw.Draw(output_img)
        
        predictions = []
        
        # For each cell, crop and classify
        for idx, (box, true_label) in enumerate(zip(boxes, true_labels)):
            if idx >= len(boxes):  # Skip padded boxes
                break
                
            x1, y1, x2, y2 = box
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            
            # Clip to image bounds
            x1 = max(x1, 0)
            y1 = max(y1, 0)
            x2 = min(x2, width)
            y2 = min(y2, height)
            
            if y2 <= y1 or x2 <= x1:
                continue
            
            # Crop cell
            cell_img = image[y1:y2, x1:x2].numpy().astype(np.uint8)
            cell_img_resized = tf.image.resize(cell_img[None], cfg.IMAGE_SIZE)[0]
            cell_img_resized = cell_img_resized / 255.0  # Normalize
            
            # Predict
            pred = model.predict(cell_img_resized[None], verbose=0)
            pred_class = np.argmax(pred[0])
            confidence = float(pred[0][pred_class])
            
            pred_name = label_map.get(int(pred_class), f'unknown({pred_class})')
            true_name = label_map.get(int(true_label), f'unknown({true_label})')
            
            # Store prediction
            predictions.append({
                'cell_id': idx + 1,
                'bbox': (x1, y1, x2, y2),
                'true_class': int(true_label),
                'true_name': true_name,
                'pred_class': int(pred_class),
                'pred_name': pred_name,
                'confidence': confidence
            })
            
            # Draw box and label
            color = 'green' if pred_class == true_label else 'red'
            draw.rectangle([(x1, y1), (x2, y2)], outline=color, width=2)
            draw.text((x1 + 5, y1 + 5), f"{pred_name}({confidence:.2f})", fill=color)
        
        # Display results
        print(f"\n📊 PREDICTIONS:")
        print(f"{'Cell':>5} {'True Label':>20} {'Predicted':>20} {'Confidence':>12} {'Match':>8}")
        print("-" * 70)
        for p in predictions:
            match = '✅' if p['true_class'] == p['pred_class'] else '❌'
            print(f"{p['cell_id']:5d} {p['true_name']:>20} {p['pred_name']:>20} {p['confidence']:>11.2%} {match:>8}")
        
        if predictions:
            accuracy = sum(1 for p in predictions if p['true_class'] == p['pred_class']) / len(predictions)
            print("-" * 70)
            print(f"Accuracy: {accuracy:.1%} ({sum(1 for p in predictions if p['true_class'] == p['pred_class'])}/{len(predictions)})")
        
        # Save output image
        output_path = '/tmp/prediction_multicell.png'
        output_img.save(output_path)
        print(f"\n💾 Saved prediction image to {output_path}")